# 邏輯迴歸的成本函數 (Cost Function for Logistic Regression)

## 目標

在這個 lab 中，你將會：

- 檢視並使用邏輯迴歸（Logistic Regression）的成本函數實作。

In [ ]:
import sys, os
import numpy as np
try:
    %matplotlib widget
except:
    %matplotlib inline
    print("Colab not support matplotlib widget")
import matplotlib.pyplot as plt
from pathlib import Path

#region 匯入資料
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import os, urllib.request, pathlib, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common_classification.py", f"{UTILS_DIR}/lab_utils_common_classification.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")


repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    

os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_common_classification import  plot_data, sigmoid, dlc

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion 


<br>

## 資料集
我們先使用與「決策邊界（Decision Boundary）」lab 相同的資料集。

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])  #(m,n)
y_train = np.array([0, 0, 0, 1, 1, 1])                                           #(m,)

我們會使用一個輔助函式來繪製資料。

- 標籤為 $y=1$ 的資料點會以紅色叉叉表示
- 標籤為 $y=0$ 的資料點會以藍色圓點表示

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

# Set both axes to be from 0-4
ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

<br>

## 成本函數

在前一個 lab 中，你已經建立了 *logistic loss* 函數。回想一下，**loss** 是針對單一訓練樣本所定義的；而在這裡，我們會把所有樣本的 loss 彙總起來，形成包含所有訓練樣本的 **cost（成本）**。

回顧邏輯迴歸的成本函數形式如下：

$$ J(\mathbf{w},b) = \frac{1}{m} \sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

其中：
- $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ 表示單一資料點的 loss：

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$

- $m$ 是資料集中訓練樣本的數量，同時：

$$
\begin{align}
  f_{\mathbf{w},b}(\mathbf{x^{(i)}}) &= g(z^{(i)})\tag{3} \\
  z^{(i)} &= \mathbf{w} \cdot \mathbf{x}^{(i)}+ b\tag{4} \\
  g(z^{(i)}) &= \frac{1}{1+e^{-z^{(i)}}}\tag{5} 
\end{align}
$$


<a name='ex-02'></a>
#### 程式碼說明

`compute_cost_logistic` 的演算法會逐一走訪所有訓練樣本：
- 對每個樣本計算對應的 loss
- 將所有樣本的 loss 累加起來

請注意，變數 X 與 y 不是純量，而是矩陣/向量：
- X 的形狀為 ($m, n$)
- y 的形狀為 ($m$,)

其中 $n$ 是特徵數量（features），$m$ 是訓練樣本數量（training examples）。


In [ ]:
def compute_cost_logistic(X, y, w, b):
    """
    Computes cost

    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """

    # YOUR CODE HERE
   
    # YOUR CODE END HERE
    return cost


請使用下方的 cell 來檢查成本函數的實作是否正確。

In [ ]:
w_tmp = np.array([1,1])
b_tmp = -3
print(compute_cost_logistic(X_train, y_train, w_tmp, b_tmp))

**預期輸出**：0.3668667864055175

<br>

## 範例
接著，我們看看在不同參數設定下，成本函數會輸出什麼結果。

- 在前一個 lab 中，你曾經繪製過 $b = -3, w_0 = 1, w_1 = 1$ 的決策邊界，也就是 `b = -3, w = np.array([1,1])`。
- 假設你想比較 $b = -4, w_0 = 1, w_1 = 1$（即 `b = -4, w = np.array([1,1])`）是否會得到更好的模型。

我們先把這兩個不同的 $b$ 值所對應的決策邊界畫出來，直覺看看哪一個比較貼合資料。

- 當 $b = -3, w_0 = 1, w_1 = 1$ 時，我們繪製 $-3 + x_0+x_1 = 0$（藍色）
- 當 $b = -4, w_0 = 1, w_1 = 1$ 時，我們繪製 $-4 + x_0+x_1 = 0$（洋紅色）

In [ ]:
import matplotlib.pyplot as plt

# Choose values between 0 and 6
x0 = np.arange(0,6)

# Plot the two decision boundaries
x1 = 3 - x0
x1_other = 4 - x0

fig,ax = plt.subplots(1, 1, figsize=(4,4))
# Plot the decision boundary
ax.plot(x0,x1, c=dlc["dlblue"], label="$b$=-3")
ax.plot(x0,x1_other, c=dlc["dlmagenta"], label="$b$=-4")
ax.axis([0, 4, 0, 4])

# Plot the original data
plot_data(X_train,y_train,ax)
ax.axis([0, 4, 0, 4])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.legend(loc="upper right")
plt.title("Decision Boundary")
plt.show()

從圖上可以看出，`b = -4, w = np.array([1,1])` 對訓練資料的擬合較差。接著我們用成本函數來驗證：實作是否也能反映出這個差異。

In [ ]:
w_array1 = np.array([1,1])
b_1 = -3
w_array2 = np.array([1,1])
b_2 = -4

print("Cost for b = -3 : ", compute_cost_logistic(X_train, y_train, w_array1, b_1))
print("Cost for b = -4 : ", compute_cost_logistic(X_train, y_train, w_array2, b_2))

**預期輸出**

Cost for b = -3 :  0.3668667864055175

Cost for b = -4 :  0.5036808636748461


可以看到成本函數的行為符合預期：`b = -4, w = np.array([1,1])` 的 cost 的確比 `b = -3, w = np.array([1,1])` 更高。

<br>

## 恭喜！
在這個 lab 中，你已經檢視並使用了邏輯迴歸的成本函數。